In [ ]:
import numpy as np
import scipy.special
from tqdm import tqdm  # Import tqdm for progress bar

class NeuralNetwork:
    def __init__(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
        self.i_nodes, self.h_nodes, self.o_nodes = input_nodes, hidden_nodes, output_nodes
        self.i_h_weight = np.random.normal(0.0, pow(self.h_nodes, -0.5), (self.h_nodes, self.i_nodes))
        self.h_o_weight = np.random.normal(0.0, pow(self.o_nodes, -0.5), (self.o_nodes, self.h_nodes))
        self.lr = learning_rate
        self.activation_function = scipy.special.expit

    def train(self, inputs_list, targets_list):
        inputs = np.array(inputs_list, ndmin=2).T
        targets = np.array(targets_list, ndmin=2).T
        hidden_outputs = self.activation_function(np.dot(self.i_h_weight, inputs))
        final_outputs = self.activation_function(np.dot(self.h_o_weight, hidden_outputs))

        output_errors = targets - final_outputs
        hidden_errors = np.dot(self.h_o_weight.T, output_errors)

        self.h_o_weight += self.lr * np.dot(output_errors * final_outputs * (1.0 - final_outputs), hidden_outputs.T)
        self.i_h_weight += self.lr * np.dot(hidden_errors * hidden_outputs * (1.0 - hidden_outputs), inputs.T)

    def query(self, inputs_list):
        inputs = np.array(inputs_list, ndmin=2).T
        hidden_outputs = self.activation_function(np.dot(self.i_h_weight, inputs))
        final_outputs = self.activation_function(np.dot(self.h_o_weight, hidden_outputs))
        return final_outputs


# Create neural network
n = NeuralNetwork(784, 200, 10, 0.1)

# Load MNIST training data
with open('/home/liy/CQU_EIE/dataEngineer/digitReco/data/mnist_train.csv', 'r') as f:
    training_data_list = f.readlines()

# Train the neural network
for e in range(1):  # Set epochs to 1 for brevity
    print(f"Epoch {e + 1}/1")
    for record in tqdm(training_data_list, desc="Training", ncols=100):
        all_values = record.split(',')
        inputs = (np.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        targets = np.zeros(10) + 0.01
        targets[int(all_values[0])] = 0.99
        n.train(inputs, targets)

# Load test data and test the neural network
with open('/home/liy/CQU_EIE/dataEngineer/digitReco/data/mnist_test.csv', 'r') as f:
    test_data_list = f.readlines()

scorecard = []
for record in test_data_list:
    all_values = record.split(',')
    correct_label = int(all_values[0])
    inputs = (np.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    label = np.argmax(n.query(inputs))
    scorecard.append(int(label == correct_label))

# Calculate and print performance score
print('Performance =', np.mean(scorecard))

In [ ]:
import cv2
path='/home/liy/CQU_EIE/dataEngineer/digitReco/test.png'
# 打开图像
image = cv2.imread(path)

# 修改图像大小
resized_image = cv2.resize(image, (200, 200))

# 保存图像
cv2.imwrite('/home/liy/CQU_EIE/dataEngineer/digitReco/output.png', resized_image)

In [ ]:
import imageio
import matplotlib.pyplot as plt
from PIL import Image

# 读取原始图像
img_array = imageio.v2.imread("/home/liy/CQU_EIE/dataEngineer/digitReco/output.png", pilmode='L')

# 打印图像形状
print("Original image shape:", img_array.shape)

# 将图像缩放到28x28
img = Image.fromarray(img_array)
img = img.resize((28, 28))
img_array = imageio.v2.imread(img, pilmode='L')

# 进行图像处理
img_shape = 255.0 - img_array.reshape(-1)
img_data = (img_shape / 255.0 * 0.99) + 0.01

# 显示图像
plt.imshow(255.0 - img_array, cmap='Greys', interpolation='None')
plt.show()

In [ ]:
# test
output = n.query(img_data)
print(output)